In [ ]:
# Sources:
# https://stackoverflow.com/questions/34714070/error-tokenizing-data-c-error-eof-following-escape-character
# https://stackoverflow.com/questions/26147180/convert-row-to-column-header-for-pandas-dataframe


# https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/

# ABOVE LINK IS IMPORTANT

# Now maybe work on API? Unless want to improve model ¯\_(ツ)_/¯ 
# Testing data set is here https://www.kaggle.com/mrisdal/fake-news

# Unfortunately have to reupload data after runtime refresh

import keras
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import nltk
nltk.download('punkt')
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from numpy import array

true_df = pd.read_csv('True.csv',  header=None)
true_df.rename(columns=true_df.iloc[0],inplace=True)
true_df = true_df.drop(true_df.index[0])


fake_df = pd.read_csv('Fake.csv',  header=None)
fake_df.rename(columns=fake_df.iloc[0],inplace=True)
fake_df= fake_df.drop(fake_df.index[0])


Vectorizer = TfidfVectorizer(stop_words="english")
TrueDF = Vectorizer.fit_transform(true_df["title"].to_list())
FakeDF = Vectorizer.fit_transform(fake_df["title"].to_list())
sentiments = []
MegaDF = []
all_words = []

#get list of all words
#first true
#sentiments are also arranged i.e true = 1, false = 0
for i in range(len(true_df)):
  blob = TextBlob(true_df.iloc[i][0])
  sentiments.append(1)
  for word in blob.words:
    all_words.append(word)
#then fake added
for i in range(len(fake_df)):
  blob = TextBlob(fake_df.iloc[i][0])
  sentiments.append(0)
  for word in blob.words:
    all_words.append(word)

#make unique
unique_words = set(all_words)
print(len(unique_words))

#make mega list containing both true and fake

for i in range(len(true_df)):
  MegaDF.append(true_df.iloc[i][0])

for i in range(len(fake_df)):
  MegaDF.append(fake_df.iloc[i][0])

#one_hot converts titles to list of word-numbers
embedded_sentences = [one_hot(sent, 40000) for sent in MegaDF]
print(embedded_sentences[0])

#lambda is basically a short function
#want to find max number of word numbers in a title
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(MegaDF, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

#the smaller ones are padded with zeroes at the end
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')


MLModel = keras.Sequential()
#MLModel.layers()
"""model.add(Dense(12, input_dim=len(TrueDF), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))"""

#40k is vocabulary. checked by printing unique word count
#60 is dimensions of word vector output
MLModel.add(keras.layers.Embedding(40000, 60, input_length=length_long_sentence))

#dk what this do
MLModel.add(keras.layers.Flatten())
MLModel.add(keras.layers.Dense(1, activation='sigmoid'))
sentiments_array = array(sentiments)
MLModel.summary()

MLModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

MLModel.fit(padded_sentences, sentiments_array, epochs=6)

#test data
fake_test = pd.read_csv('fake.csv')

#remove rows where title is NaN
fake_test.dropna(subset=['title'], inplace=True)

#turn to list
fake_list = fake_test['title'].tolist()

#test data is all fake
y = [0]*len(fake_list)

#turn to numpy array list no good
y_array = array(y)

#one_hot converts sentence to list of word-numbers
embedded_sentences2 = [one_hot(sent, 40000) for sent in fake_list]

#pad with zeroes to max len
padded_sentences2 = pad_sequences(embedded_sentences2, length_long_sentence, padding='post')

_, accuracy = MLModel.evaluate(padded_sentences2, y_array, verbose=0)
print(accuracy)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


KeyboardInterrupt: ignored

# V2 -> Feed forward
***

In [10]:
# Sources:
# https://stackoverflow.com/questions/34714070/error-tokenizing-data-c-error-eof-following-escape-character
# https://stackoverflow.com/questions/26147180/convert-row-to-column-header-for-pandas-dataframe
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
# https://www.youtube.com/watch?v=bKLL0tAj3GE&ab_channel=BhaveshBhatt
# https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python
# programiz.com/python-programming/json
# https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

# Focus mainly on headlines
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split

# Set up data frames
true_df = pd.read_csv('True.csv',  header=None)
true_df.rename(columns=true_df.iloc[0],inplace=True)
true_df = true_df.drop(true_df.index[0])
TrueCol = [1 for i in range(true_df.shape[0])]
true_df["Label"] = TrueCol

fake_df = pd.read_csv('Fake.csv',  header=None)
fake_df.rename(columns=fake_df.iloc[0],inplace=True)
fake_df= fake_df.drop(fake_df.index[0])
FakeCol = [0 for i in range(fake_df.shape[0])]
fake_df["Label"] = FakeCol

# Prepare Training Set
TrainingSet  = pd.concat([fake_df,true_df])
TrainingSet  = TrainingSet.sample(frac=1).reset_index(drop=True)
TrainingSet = TrainingSet.drop(axis=1,columns=["subject","date","text"])
TrainingSet = TrainingSet.loc[:10000,:]

# Vectorize with bag of words model
Vectorizer = TfidfVectorizer(stop_words="english")
TokenizedText = Vectorizer.fit_transform(TrainingSet["title"].to_list()).toarray().tolist()
# Setup Model
Model = Sequential()
Model.add(Dense(80, input_dim=len(TokenizedText[0]), activation='relu'))
Model.add(Dense(40, activation='relu'))
Model.add(Dense(40, activation='relu'))
Model.add(Dense(1, activation='sigmoid'))

# Compile and train
Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
XTrain, XTest, YTrain, YTest = train_test_split(TokenizedText,TrainingSet["Label"].tolist(), test_size=0.3, random_state=21)
Model.fit(XTrain,YTrain,batch_size=100,epochs=20)


Epoch 1/20
70/70 [==============================] - 1s 4ms/step - loss: 0.6537 - accuracy: 0.7295
Epoch 2/20
70/70 [==============================] - 0s 4ms/step - loss: 0.1406 - accuracy: 0.9642
Epoch 3/20
70/70 [==============================] - 0s 4ms/step - loss: 0.0223 - accuracy: 0.9954
Epoch 4/20
70/70 [==============================] - 0s 4ms/step - loss: 0.0069 - accuracy: 0.9993
Epoch 5/20
70/70 [==============================] - 0s 4ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 6/20
70/70 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 7/20
70/70 [==============================] - 0s 4ms/step - loss: 7.4592e-04 - accuracy: 1.0000
Epoch 8/20
70/70 [==============================] - 0s 4ms/step - loss: 5.3412e-04 - accuracy: 1.0000
Epoch 9/20
70/70 [==============================] - 0s 4ms/step - loss: 3.9937e-04 - accuracy: 1.0000
Epoch 10/20
70/70 [==============================] - 0s 4ms/step - loss: 3.0572e-04 - accuracy: 1.0000
Epo

In [4]:
# Get predictions
PredictionResults = Model.predict(XTest)

In [5]:
# Get predication results(normalized)
TestArray = []
for i in PredictionResults:
  if i >= 0.5:
    TestArray.append(1)
  else:
    TestArray.append(0)

  

In [6]:
from sklearn.metrics import accuracy_score

# Good in da hood
print("Accurracy: ~{}".format(round(accuracy_score(YTest,TestArray)*100)))


Accurracy: ~91


In [11]:
# Convert to Tensorflow lite for website purposes
from tensorflow import lite
Converter = lite.TFLiteConverter.from_keras_model(Model)
TFLiteModel = Converter.convert()
open("MainMode.tflite","wb").write(TFLiteModel)

INFO:tensorflow:Assets written to: /tmp/tmp4_l43j34/assets


3928124

In [ ]:
# Test TFLite model (thing that will go to the website)
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path="MainMode.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on random input data.
input_shape = input_details[0]['shape']

for i in range(len(XTest)):
  input_data = np.array(XTest[i:i+1], dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  Result = output_data[0][0]
  if Result >= 0.5:
    print(1)
  else:
    print(0)



In [12]:

pickle.dump(Vectorizer,open("Vectorizer.pickle","wb"))